In [8]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
from PIL import Image
import os
from collections import defaultdict

def get_class_names(data_dir):
    return sorted([d for d in os.listdir(data_dir) 
                  if os.path.isdir(os.path.join(data_dir, d))])

def load_trained_model(model_path, num_classes):
    model = models.resnet18(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    model.load_state_dict(torch.load(model_path))
    return model

def get_test_transform():
    return transforms.Compose([
        transforms.Resize(512),
        transforms.CenterCrop(512),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                           [0.229, 0.224, 0.225])
    ])

def predict_single_image(model, image_path, transform, device):
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(image_tensor)
        _, predicted = torch.max(outputs, 1)
        probabilities = torch.nn.functional.softmax(outputs, dim=1)
    
    return predicted.item(), probabilities[0]

def test_campus_vision(model_path, test_data_dir):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    
    class_names = get_class_names(test_data_dir)
    num_classes = len(class_names)
    print(f"Found {num_classes} classes: {', '.join(class_names)}")
    print("Starting evaluation...\n")
    
    model = load_trained_model(model_path, num_classes)
    model = model.to(device)
    model.eval()
    
    transform = get_test_transform()
    results = []
    total_images = 0
    total_correct = 0
    
    for class_name in class_names:
        class_dir = os.path.join(test_data_dir, class_name)
        print(f"Processing {class_name}...", end='\r')
        
        correct = 0
        total = 0
        misclassifications = defaultdict(list)  # Store misclassified images by predicted class
        
        image_files = [f for f in os.listdir(class_dir) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for image_name in image_files:
            image_path = os.path.join(class_dir, image_name)
            predicted_class_idx, probabilities = predict_single_image(
                model, image_path, transform, device
            )
            
            predicted_class = class_names[predicted_class_idx]
            confidence = probabilities[predicted_class_idx].item() * 100
            
            total += 1
            if predicted_class == class_name:
                correct += 1
            else:
                # Store misclassified image name and confidence
                misclassifications[predicted_class].append((image_name, confidence))
                
                # Print misclassified file information immediately
                print(f"Misclassified {image_name} as {predicted_class} with confidence {confidence:.2f}%")
        
        accuracy = (correct / total) * 100
        results.append((class_name, accuracy, total))
        total_images += total
        total_correct += correct
        
        # Print class summary
        print(f"\nClass: {class_name}")
        print(f"Total images: {total}")
        print(f"Correct predictions: {correct}")
        print(f"Accuracy: {accuracy:.2f}%")
        
        if misclassifications:
            print("\nMisclassified images:")
            for pred_class, images in misclassifications.items():
                print(f"\n  Predicted as {pred_class} ({len(images)} images):")
                for img_name, conf in images:
                    print(f"    - {img_name} (confidence: {conf:.2f}%)")
        else:
            print("\nNo misclassifications!")
            
        print("-" * 50)
    
    # Print final summary
    print("\nFinal Testing Summary:")
    print("=" * 50)
    print(f"Total number of classes: {num_classes}")
    print(f"Total number of images: {total_images}")
    print(f"Overall accuracy: {(total_correct/total_images)*100:.2f}%")
    print("\nPer-class Results:")
    print("-" * 50)
    
    # Sort results by accuracy
    results.sort(key=lambda x: x[1], reverse=True)
    for class_name, accuracy, count in results:
        print(f"{class_name}:")
        print(f"  Total images: {count}")
        print(f"  Accuracy: {accuracy:.2f}%")
    
    return results

if __name__ == "__main__":
    MODEL_PATH = "models/resnet-pretrained-large.pth"
    CAMPUS_VISION_DIR = "campusVision"
    
    results = test_campus_vision(MODEL_PATH, CAMPUS_VISION_DIR)


Using device: cpu
Found 10 classes: Butler, Carpenter, Lee, McCain, McCool, OldMain, Simrall, Swalm, Union, Walker
Starting evaluation...

Processing Butler...
Class: Butler
Total images: 1267
Correct predictions: 1267
Accuracy: 100.00%

No misclassifications!
--------------------------------------------------
Misclassified Carpenter_Hall_4.png as Lee with confidence 72.53%
Misclassified Carpenter_Hall_19.png as Simrall with confidence 51.05%
Misclassified Carpenter_Hall_716.png as McCain with confidence 30.56%
Misclassified Carpenter_Hall_731.png as Lee with confidence 50.19%
Misclassified Carpenter_Hall_734.png as McCain with confidence 36.81%
Misclassified Carpenter_Hall_738.png as Swalm with confidence 50.83%
Misclassified Carpenter_Hall_770.png as Lee with confidence 50.50%
Misclassified Carpenter_Hall_772.png as Lee with confidence 99.13%
Misclassified Carpenter_Hall_986.png as Lee with confidence 50.69%
Misclassified Carpenter_Hall_1008.png as Lee with confidence 86.27%
Misclass

KeyboardInterrupt: 